In [1]:
import os

# Measure Package 0: Baseline
menu_mp = 0
input_mp = 'baseline'

# Print verbsose output variable
print_debug = True

# import from cmu-tare-model package
from config import PROJECT_ROOT
import pandas as pd

# Set columns in display
# pd.set_option('display.max_columns', None)
# pd.reset_option('display.max_columns') # Reset options to default

# Set rows in display
# pd.set_option('display.max_rows', None)
# pd.reset_option('display.max_rows') # Reset options to default

# import seaborn library (wrapper of matplotlib)
import seaborn as sns
sns.set_theme(style="darkgrid")

from datetime import datetime

# Get the current datetime
# Start the timer
start_time = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")

# Storing Result Outputs in output_results folder
relative_path = os.path.join("cmu_tare_model", "output_results")
output_folder_path = os.path.join(PROJECT_ROOT, relative_path)

print(f"""
=========================================================================================================
Welcome to the Trade-off Analysis of residential Retrofits for energy Equity Tool (TARE Model)
Let's start by reading the data from the NREL EUSS Database.

Make sure that the zipped folders stay organized as they are once unzipped.
If changes are made to the file path, then the program will not run properly.
==========================================================================================================
      
Project root directory: {PROJECT_ROOT}
Result outputs will be exported here: {output_folder_path}

""")

Project root directory: c:\users\14128\research\cmu-tare-model

Welcome to the Trade-off Analysis of residential Retrofits for energy Equity Tool (TARE Model)
Let's start by reading the data from the NREL EUSS Database.

Make sure that the zipped folders stay organized as they are once unzipped.
If changes are made to the file path, then the program will not run properly.
      
Project root directory: c:\users\14128\research\cmu-tare-model
Result outputs will be exported here: c:\users\14128\research\cmu-tare-model\cmu_tare_model\output_results




# LOAD EUSS DATA: Annual Energy Consumption and Metadata
## BASELINE (MP0): Metadata, Space Heating, Water Heating, Clothes Drying, Cooking

In [2]:
from cmu_tare_model.energy_consumption_and_metadata.user_input_geographic_filter import *
from cmu_tare_model.energy_consumption_and_metadata.process_euss_data import *

print(F"""
====================================================================================================================================================================
LOAD EUSS DATA FOR BASELINE SCENARIO: MEASURE PACKAGE {menu_mp} (MP{menu_mp})
====================================================================================================================================================================
Load data from the NREL EUSS Database for the baseline scenario and apply various filters.

DATA: NREL EUSS Database
HOUSING FILTERS: Occupied units and Single Family Homes
GEOGRAPHIC FILTERS: National, State, or City

Additional details data can be found in the official End-Use Load Profiles/Savings Shapes documentation.
Our methodology is detailed in the process_euss_data.py file and its associated imports.

RESULTS OUTPUT:
""")

# Measure Package 0: Baseline
filename = "baseline_metadata_and_annual_results.csv"
relative_path = os.path.join("cmu_tare_model", "data", "euss_data", "resstock_amy2018_release_1.1", "state", filename)

file_path = os.path.join(PROJECT_ROOT, relative_path)

print(f"Retrieved data for filename: {filename}")
print(f"Located at filepath: {file_path}")

# Fix DtypeWarning error in columns:
# 'in.neighbors', 'in.geometry_stories_low_rise', 'in.iso_rto_region', 'in.pv_orientation', 'in.pv_system_size'
columns_to_string = {11: str, 61: str, 121: str, 103: str, 128: str, 129: str}
df_euss_am_baseline = pd.read_csv(file_path, dtype=columns_to_string, index_col="bldg_id") # UPDATE: Set index to 'bldg_id'

# Filter for occupied homes
occupancy_filter = df_euss_am_baseline['in.vacancy_status'] == 'Occupied'
df_euss_am_baseline = df_euss_am_baseline.loc[occupancy_filter]

# Filter for single family home building type
house_type_list = ['Single-Family Attached', 'Single-Family Detached']
house_type_filter = df_euss_am_baseline['in.geometry_building_type_recs'].isin(house_type_list)
df_euss_am_baseline = df_euss_am_baseline.loc[house_type_filter]

# Choose between national or sub-national level analysis
menu_state = get_menu_choice(menu_prompt, {'N', 'Y'})   # This code is only run in baseline

# National Level 
if menu_state == 'N':
    print("You chose to analyze all of the United States.")
    input_state = 'National'
    location_id = 'National'

# Filter down to state or city
else:
    input_state = get_state_choice(df_euss_am_baseline)    
    print(f"You chose to filter for: {input_state}")

    location_id = str(input_state)
    print(f"Location ID is: {location_id}")


    state_filter = df_euss_am_baseline['in.state'].eq(input_state)
    df_euss_am_baseline = df_euss_am_baseline.loc[state_filter]

    print(city_prompt)
    print(df_euss_am_baseline['in.city'].value_counts())

    menu_city = get_menu_choice(city_menu_prompt, {'N', 'Y'})

    # Filter for the entire selected state
    if menu_city == 'N':
        print(f"You chose to analyze all of state: {input_state}")
        
        location_id = str(input_state)
        print(f"Location ID is: {location_id}")
        
    # Filter to a city within the selected state
    else:
        input_cityFilter = get_city_choice(df_euss_am_baseline, input_state)
        print(f"You chose to filter for: {input_state}, {input_cityFilter}")

        location_id = input_cityFilter.replace(', ', '_').strip()
        print(f"Location ID is: {location_id}")

        city_filter = df_euss_am_baseline['in.city'].eq(f"{input_state}, {input_cityFilter}")
        df_euss_am_baseline = df_euss_am_baseline.loc[city_filter]

print(f"""
====================================================================================================================================================================      
DATAFRAME: df_euss_am_baseline
      
{df_euss_am_baseline}
""")


LOAD EUSS DATA FOR BASELINE SCENARIO: MEASURE PACKAGE 0 (MP0)
Load data from the NREL EUSS Database for the baseline scenario and apply various filters.

DATA: NREL EUSS Database
HOUSING FILTERS: Occupied units and Single Family Homes
GEOGRAPHIC FILTERS: National, State, or City

Additional details data can be found in the official End-Use Load Profiles/Savings Shapes documentation.
Our methodology is detailed in the process_euss_data.py file and its associated imports.

RESULTS OUTPUT:

Retrieved data for filename: baseline_metadata_and_annual_results.csv
Located at filepath: c:\users\14128\research\cmu-tare-model\cmu_tare_model\data\euss_data\resstock_amy2018_release_1.1\state\baseline_metadata_and_annual_results.csv
You chose to filter for: PA
Location ID is: PA

To accurately characterize load profile, it is recommended to select subsets of data with >= 1000 models (~240,000 representative dwelling units).

The following cities (number of models also shown) are available for this 

In [3]:
print(F"""
====================================================================================================================================================================
LOAD EUSS DATA FOR BASELINE SCENARIO: MEASURE PACKAGE {menu_mp} (MP{menu_mp})
====================================================================================================================================================================
In addition to the housing type and occupancy filters, the data has been filtered for fuel and technology
Please see the energy_consumption_and_metadata folder for more details on data processing.

We employ the df_enduse_refactored function to process the data and create a new dataframe, which is part of the 5-step data validation process:
1. Mask Initialization with initialize_validation_tracking()
2. Series Initialization with create_retrofit_only_series()
3. Valid-Only Calculation for qualifying homes
4. Valid-Only Updates using list-based collection
5. Final Masking with apply_final_masking()

Documentation for df_enduse_refactored function:
{df_enduse_refactored.__doc__}
----------------------------------------------------------------

RESULTS OUTPUT (BASELINE):
""")

# df_baseline_enduse(df_baseline, df_enduse, category, fuel_filter='Yes', tech_filter='Yes')
df_euss_am_baseline_home = df_enduse_refactored(
    df_baseline = df_euss_am_baseline
    )

print(f"""
====================================================================================================================================================================
DATAFRAME: df_euss_am_baseline_home
      
{df_euss_am_baseline_home}
""")


LOAD EUSS DATA FOR BASELINE SCENARIO: MEASURE PACKAGE 0 (MP0)
In addition to the housing type and occupancy filters, the data has been filtered for fuel and technology
Please see the energy_consumption_and_metadata folder for more details on data processing.

We employ the df_enduse_refactored function to process the data and create a new dataframe, which is part of the 5-step data validation process:
1. Mask Initialization with initialize_validation_tracking()
2. Series Initialization with create_retrofit_only_series()
3. Valid-Only Calculation for qualifying homes
4. Valid-Only Updates using list-based collection
5. Final Masking with apply_final_masking()

Documentation for df_enduse_refactored function:
Creates a standardized energy usage DataFrame and applies data quality filters.

    This function creates a new DataFrame with standardized column names and structure,
    calculates total consumption by fuel type, creates data quality flags for analysis,
    and sets invalid cons

# Project Future Energy ConsumptioN
## Assumptions:
- Using EIA Heating Degree Day (HDD) Forecasted Data (Factors)
- Originally projected both space heating and water heating, now just space heating.

In [4]:
from cmu_tare_model.energy_consumption_and_metadata.project_future_energy_consumption import *

print(f"""
====================================================================================================================================================================
PROJECT FUTURE ENERGY CONSUMPTION: BASELINE SCENARIO
====================================================================================================================================================================
project_future_energy_consumption.py file was used to calculate/project the annual energy consumption 
for each home in the dataframe.
      
Documentation for project_future_energy_consumption function:
{project_future_consumption.__doc__}

RESULTS OUTPUT (BASELINE):
""")

# Make a copy of the dataframe
df_euss_am_baseline_home = df_euss_am_baseline_home.copy()
df_baseline_scenario_consumption = df_euss_am_baseline_home.copy()

# Project Future Energy Consumption
df_euss_am_baseline_home, df_baseline_scenario_consumption = project_future_consumption(
    df=df_euss_am_baseline_home,
    menu_mp=menu_mp,
    base_year=2024
    )

Retrieved data for filename: aeo_projections_2022_2050.xlsx
Located at filepath: c:\users\14128\research\cmu-tare-model\cmu_tare_model\data\projections\aeo_projections_2022_2050.xlsx

PROJECT FUTURE ENERGY CONSUMPTION: BASELINE SCENARIO
project_future_energy_consumption.py file was used to calculate/project the annual energy consumption 
for each home in the dataframe.
      
Documentation for project_future_energy_consumption function:
Projects future energy consumption based on baseline or upgraded equipment specifications.

    This function calculates projected energy consumption for each equipment category (heating,
    water heating, clothes drying, and cooking) for future years based on their expected lifetime.
    It applies climate adjustment factors for heating and water heating categories.

    Args:
        df: The input DataFrame containing baseline consumption data.
        menu_mp: Indicates the measure package to apply. 0 for baseline, or a positive integer
            

In [5]:
print(f"""
====================================================================================================================================================================
      
DATAFRAME: df_euss_am_baseline_home

{df_euss_am_baseline_home}      

DATAFRAME: df_baseline_scenario_consumption
      
{df_baseline_scenario_consumption}
""")

if print_debug:
    from cmu_tare_model.utils.create_sample_df import create_sample_df

    df_sample_heating = create_sample_df(
        df=df_euss_am_baseline_home,
        include_groups=['base_equipment'],
        categories=['heating'],
        # scenarios=['in.annual_energy_kwh'],
        # metrics=['in.annual_energy_kwh'],
        # mp_number=menu_mp,
        regex_patterns=['valid_fuel_heating', 'valid_tech_heating', 'include_heating']
    )
    print(df_sample_heating)


    df_sample_waterHeating = create_sample_df(
        df=df_euss_am_baseline_home,
        include_groups=['base_equipment', 'consumption',],
        categories=['waterHeating'],
        # scenarios=['in.annual_energy_kwh'],
        # metrics=['in.annual_energy_kwh'],
        # mp_number=menu_mp,
        regex_patterns=['valid_fuel_waterHeating', 'valid_tech_waterHeating', 'include_waterHeating']
    )
    print(df_sample_waterHeating)


    df_sample_clothesDrying = create_sample_df(
        df=df_euss_am_baseline_home,
        include_groups=['base_equipment', 'consumption',],
        categories=['clothesDrying'],
        # scenarios=['in.annual_energy_kwh'],
        # metrics=['in.annual_energy_kwh'],
        # mp_number=menu_mp,
        regex_patterns=['valid_fuel_clothesDrying', 'valid_tech_clothesDrying', 'include_clothesDrying']
    )
    print(df_sample_clothesDrying)


    df_sample_cooking = create_sample_df(
        df=df_euss_am_baseline_home,
        include_groups=['base_equipment', 'consumption',],
        categories=['cooking'],
        # scenarios=['in.annual_energy_kwh'],
        # metrics=['in.annual_energy_kwh'],
        # mp_number=menu_mp,
        regex_patterns=['valid_fuel_cooking', 'valid_tech_cooking', 'include_cooking']
    )
    print(df_sample_cooking)



      
DATAFRAME: df_euss_am_baseline_home

         square_footage census_region  census_division census_division_recs  \
bldg_id                                                                       
119              2152.0     Northeast  Middle Atlantic      Middle Atlantic   
122              2176.0     Northeast  Middle Atlantic      Middle Atlantic   
150              1690.0     Northeast  Middle Atlantic      Middle Atlantic   
153              2176.0     Northeast  Middle Atlantic      Middle Atlantic   
162              2663.0     Northeast  Middle Atlantic      Middle Atlantic   
...                 ...           ...              ...                  ...   
549882           1202.0     Northeast  Middle Atlantic      Middle Atlantic   
549915           2176.0     Northeast  Middle Atlantic      Middle Atlantic   
549937            885.0     Northeast  Middle Atlantic      Middle Atlantic   
549963           1690.0     Northeast  Middle Atlantic      Middle Atlantic   
549989 

# PUBLIC IMPACTS (BASELINE): Climate and Health Damages

In [6]:
from cmu_tare_model.public_impact.calculate_lifetime_climate_impacts_sensitivity import *
from cmu_tare_model.public_impact.calculate_lifetime_health_impacts_sensitivity import *

print(f"""
====================================================================================================================================================================
PUBLIC IMPACTS (BASELINE): DAMAGES FROM CLIMATE AND HEALTH-RELATED EMISSIONS
====================================================================================================================================================================
Detailed documentation of the methods used to calculate the climate and public health impacts can be
found in the public_impacts folder of the cmu-tare-model package. All monetary values are in 2023
inflation-adjusted dollars ($USD2023).
- data_processing sub-folder: contains the data processing scripts used for lookup dictionaries
- calculations sub-folder: contains the calculations for emissions from fossul fuel appliances
- Main folder contains the scripts for calculating lifetime climate and public health impacts as
  as well as climate NPV, health NPV, and combined public NPV.

Step 1: Calculate the baseline marginal damages for climate and health-related emissions
Step 2: Calculate the post-retrofit marginal damages for climate and health-related emissions
Step 3: Discount climate and health impacts and calculate lifetime public impacts (public NPV)

----------------------------------------------------------------------------------------------
Step 1: Calculate the baseline marginal damages for climate and health-related emissions
----------------------------------------------------------------------------------------------

Documentation for calculate_lifetime_climate_impacts_sensitivity function:
{calculate_lifetime_climate_impacts.__doc__}
      
Documentation for calculate_lifetime_health_impacts_sensitivity function:
{calculate_lifetime_health_impacts.__doc__}

""")

Fuel sources in df_grid_mix: {'Renewable', 'Coal', 'Oil', 'Natural Gas', 'Nuclear'}
Fuel sources in df_grid_emis_factors: {'Coal', 'Oil', 'Natural Gas', 'Nuclear', 'Renewables'}
Retrieved data for filename: bls_cpiu_2005-2023.xlsx
Located at filepath: c:\users\14128\research\cmu-tare-model\cmu_tare_model\data\inflation_data\bls_cpiu_2005-2023.xlsx
    year  cpiu_annual
0   2005      195.300
1   2006      201.600
2   2007      207.342
3   2008      215.303
4   2009      214.537
5   2010      218.056
6   2011      224.939
7   2012      229.594
8   2013      232.957
9   2014      236.736
10  2015      237.017
11  2016      240.007
12  2017      245.120
13  2018      251.107
14  2019      255.657
15  2020      258.811
16  2021      270.970
17  2022      292.655
18  2023      304.702

Calculating state-level averages for fallback...
Added state-level fallbacks for 49 states, 3 models, and 3 pollutants:
- States: AL, AR, AZ, CA, CO, CT, DC, DE, FL, GA, IA, ID, IL, IN, KS, KY, LA, MA, MD, ME,

## PUBLIC IMPACTS (BASELINE): Climate Damages

In [7]:
# Make copies to prevent overwriting the original dataframe and compare the differences
df_euss_am_baseline_home = df_euss_am_baseline_home.copy()
df_baseline_damages_climate = df_euss_am_baseline_home.copy()

print(f"""
==================== PUBLIC IMPACTS (BASELINE): DAMAGES FROM CLIMATE-RELATED EMISSIONS ====================

""")

# Modified usage pattern - keeping df_euss_am_baseline_home as a single DataFrame
# while keeping detailed results separate
df_euss_am_baseline_home, df_baseline_damages_climate = calculate_lifetime_climate_impacts(
    df=df_euss_am_baseline_home,
    menu_mp=0,  # baseline
    policy_scenario='No Inflation Reduction Act',
    verbose=True  # Add this parameter
)


==================== PUBLIC IMPACTS (BASELINE): DAMAGES FROM CLIMATE-RELATED EMISSIONS ====================


-- Scenario: Baseline -- 
              scenario_prefix: 'baseline_', cambium_scenario: 'MidCase', lookup_emissions_fossil_fuel: 'lookup_emissions_fossil_fuel', 
              lookup_emissions_electricity_climate: 'lookup_emissions_electricity_climate_preIRA', lookup_emissions_electricity_health: 'lookup_emissions_electricity_health', lookup_fuel_prices: 'lookup_fuel_prices_preIRA'
              
Calculating Climate Emissions and Damages from 2024 to 2039 for heating
Baseline calculation for heating:
  - 12266 homes have valid data
  - 3385 homes have invalid data (values will be NaN)
Calculating Climate Emissions and Damages from 2024 to 2036 for waterHeating
Baseline calculation for waterHeating:
  - 14999 homes have valid data
  - 652 homes have invalid data (values will be NaN)
Calculating Climate Emissions and Damages from 2024 to 2037 for clothesDrying
Baseline calculati

In [8]:
# Climate Change Impacts: Baseline Scenario
print(f"""
====================================================================================================================================================================

df_euss_am_baseline_home: DataFrame containing the baseline scenario data
{df_euss_am_baseline_home}

df_baseline_damages_climate: DataFrame containing the baseline scenario data with climate damages      
{df_baseline_damages_climate}

""")

if print_debug:
    # df_euss_am_baseline_home
    print(f"Shape of df_euss_am_baseline_home: {df_euss_am_baseline_home.shape}")

    # Print columns that contain the word "damages"
    damage_columns = [col for col in df_euss_am_baseline_home.columns if "damages" in col.lower()]
    print("\nColumns containing 'damages':")
    print(damage_columns)

    # df_baseline_damages_climate
    print(f"Shape of df_baseline_damages_climate: {df_baseline_damages_climate.shape}")

    # Print columns that contain the word "damages"
    damage_columns = [col for col in df_baseline_damages_climate.columns if "damages" in col.lower()]
    print("\nColumns containing 'damages':")
    print(damage_columns)



df_euss_am_baseline_home: DataFrame containing the baseline scenario data
         square_footage census_region  census_division census_division_recs  \
bldg_id                                                                       
119              2152.0     Northeast  Middle Atlantic      Middle Atlantic   
122              2176.0     Northeast  Middle Atlantic      Middle Atlantic   
150              1690.0     Northeast  Middle Atlantic      Middle Atlantic   
153              2176.0     Northeast  Middle Atlantic      Middle Atlantic   
162              2663.0     Northeast  Middle Atlantic      Middle Atlantic   
...                 ...           ...              ...                  ...   
549882           1202.0     Northeast  Middle Atlantic      Middle Atlantic   
549915           2176.0     Northeast  Middle Atlantic      Middle Atlantic   
549937            885.0     Northeast  Middle Atlantic      Middle Atlantic   
549963           1690.0     Northeast  Middle Atlantic 

## PUBLIC IMPACTS (BASELINE): Health Damages

In [9]:
# Make copies to prevent overwriting the original dataframe and compare the differences
df_euss_am_baseline_home = df_euss_am_baseline_home.copy()
df_baseline_damages_health = df_euss_am_baseline_home.copy()

print(f"""
==================== PUBLIC IMPACTS (BASELINE): DAMAGES FROM HEALTH-RELATED EMISSIONS ====================

""")

df_euss_am_baseline_home, df_baseline_damages_health = calculate_lifetime_health_impacts(
    df=df_euss_am_baseline_home,
    menu_mp=0,  # baseline
    policy_scenario='No Inflation Reduction Act',
    debug=True,
    verbose=True  # Add this parameter
)


==================== PUBLIC IMPACTS (BASELINE): DAMAGES FROM HEALTH-RELATED EMISSIONS ====================



HEALTH IMPACT DATA COVERAGE ANALYSIS

Analyzed 67 counties across 1 states

FOSSIL FUEL HEALTH IMPACT COVERAGE:
----------------------------------------
Model: ap2, CR Function: acs
  - County-level data: 67/67 counties (100.0%)
  - State-level fallback: 0/67 counties (0.0%)
  - Missing data: 0/67 counties (0.0%)
  - Total coverage: 67/67 counties (100.0%)
Model: ap2, CR Function: h6c
  - County-level data: 67/67 counties (100.0%)
  - State-level fallback: 0/67 counties (0.0%)
  - Missing data: 0/67 counties (0.0%)
  - Total coverage: 67/67 counties (100.0%)
Model: easiur, CR Function: acs
  - County-level data: 67/67 counties (100.0%)
  - State-level fallback: 0/67 counties (0.0%)
  - Missing data: 0/67 counties (0.0%)
  - Total coverage: 67/67 counties (100.0%)
Model: easiur, CR Function: h6c
  - County-level data: 67/67 counties (100.0%)
  - State-level fallback: 0/67 count

In [10]:
# Health Impacts: Baseline Scenario
print(f"""
====================================================================================================================================================================
df_euss_am_baseline_home: DataFrame containing the baseline scenario data
{df_euss_am_baseline_home}
      
df_baseline_damages_health: DataFrame containing the baseline scenario data with health damages
{df_baseline_damages_health}

""")

if print_debug:
    # df_euss_am_baseline_home
    print(f"Shape of df_euss_am_baseline_home: {df_euss_am_baseline_home.shape}")

    # Print columns that contain the word "damages"
    damage_columns = [col for col in df_euss_am_baseline_home.columns if "damages" in col.lower()]
    print("\nColumns containing 'damages':")
    print(damage_columns)

    # df_baseline_damages_health
    print(f"Shape of df_baseline_damages_health: {df_baseline_damages_health.shape}")

    # Print columns that contain the word "damages"
    damage_columns = [col for col in df_baseline_damages_health.columns if "damages" in col.lower()]
    print("\nColumns containing 'damages':")
    print(damage_columns)


df_euss_am_baseline_home: DataFrame containing the baseline scenario data
         square_footage census_region  census_division census_division_recs  \
bldg_id                                                                       
119              2152.0     Northeast  Middle Atlantic      Middle Atlantic   
122              2176.0     Northeast  Middle Atlantic      Middle Atlantic   
150              1690.0     Northeast  Middle Atlantic      Middle Atlantic   
153              2176.0     Northeast  Middle Atlantic      Middle Atlantic   
162              2663.0     Northeast  Middle Atlantic      Middle Atlantic   
...                 ...           ...              ...                  ...   
549882           1202.0     Northeast  Middle Atlantic      Middle Atlantic   
549915           2176.0     Northeast  Middle Atlantic      Middle Atlantic   
549937            885.0     Northeast  Middle Atlantic      Middle Atlantic   
549963           1690.0     Northeast  Middle Atlantic  

# PRIVATE IMPACTS (BASELINE): Lifetime Fuel Costs

In [11]:
from cmu_tare_model.private_impact.calculate_lifetime_fuel_costs import *

print(f"""
=====================================================================================================================================================================
LIFETIME FUEL COSTS: Baseline Scenario
=====================================================================================================================================================================
STEPS
- Create a mapping dictionary for fuel types
- Create new merge columns to ensure a proper match.
- Merge df_copy with df_fuel_prices to get fuel prices for electricity, natural gas, propane, and fuel oil
- Calculate the per kWh fuel costs for each fuel type and region
- Calculate the baseline fuel cost 

----------------------------------------------------------------------------------------------------------------------
Step 1: Calculate annual operating (fuel) costs
----------------------------------------------------------------------------------------------------------------------

Documentation for calculate_lifetime_fuel_costs function:
{calculate_lifetime_fuel_costs.__doc__}

====================================================================================================================================================================
FUEL COSTS RESULTS (BASELINE): 

""")

# Make copies to prevent overwriting the original dataframe and compare the differences
df_euss_am_baseline_home = df_euss_am_baseline_home.copy()
df_baseline_fuel_costs = df_euss_am_baseline_home.copy()

# def calculate_lifetime_climate_impacts(df: pd.DataFrame, menu_mp: int, policy_scenario: str, df_baseline_damages: Optional[pd.DataFrame] = None) -> Tuple[pd.DataFrame, pd.DataFrame]:
# Returns df_main, df_detailed
df_euss_am_baseline_home, df_baseline_fuel_costs = calculate_lifetime_fuel_costs(
    df=df_euss_am_baseline_home,
    menu_mp=menu_mp,
    policy_scenario='No Inflation Reduction Act'
    )


LIFETIME FUEL COSTS: Baseline Scenario
STEPS
- Create a mapping dictionary for fuel types
- Create new merge columns to ensure a proper match.
- Merge df_copy with df_fuel_prices to get fuel prices for electricity, natural gas, propane, and fuel oil
- Calculate the per kWh fuel costs for each fuel type and region
- Calculate the baseline fuel cost 

----------------------------------------------------------------------------------------------------------------------
Step 1: Calculate annual operating (fuel) costs
----------------------------------------------------------------------------------------------------------------------

Documentation for calculate_lifetime_fuel_costs function:

    Calculate lifetime fuel costs for each equipment category.

    This function processes each equipment category over its lifetime, computing annual
    and lifetime fuel costs. Results are combined into two DataFrames:
    a main summary (df_main) and a detailed annual breakdown (df_detailed).
  

In [12]:
print(f"""
====================================================================================================================================================================

df_euss_am_baseline_home: DataFrame containing the baseline scenario data
{df_euss_am_baseline_home}
      
df_baseline_fuel_costs: DataFrame containing the baseline scenario data with fuel costs
{df_baseline_fuel_costs}

""")

if print_debug:
    # df_euss_am_baseline_home
    print(f"Shape of df_euss_am_baseline_home: {df_euss_am_baseline_home.shape}")

    # Print columns that contain the word "fuel_cost"
    fuel_cost_columns = [col for col in df_euss_am_baseline_home.columns if "fuel_cost" in col]
    print("\nColumns containing 'fuel_cost':")
    print(fuel_cost_columns)

    # df_baseline_fuel_costs
    print(f"Shape of df_baseline_fuel_costs: {df_baseline_fuel_costs.shape}")

    # Print columns that contain the word "damages"
    fuel_cost_columns = [col for col in df_baseline_fuel_costs.columns if "fuel_cost" in col]
    print("\nColumns containing 'fuel_cost':")
    print(fuel_cost_columns)



df_euss_am_baseline_home: DataFrame containing the baseline scenario data
         square_footage census_region  census_division census_division_recs  \
bldg_id                                                                       
119              2152.0     Northeast  Middle Atlantic      Middle Atlantic   
122              2176.0     Northeast  Middle Atlantic      Middle Atlantic   
150              1690.0     Northeast  Middle Atlantic      Middle Atlantic   
153              2176.0     Northeast  Middle Atlantic      Middle Atlantic   
162              2663.0     Northeast  Middle Atlantic      Middle Atlantic   
...                 ...           ...              ...                  ...   
549882           1202.0     Northeast  Middle Atlantic      Middle Atlantic   
549915           2176.0     Northeast  Middle Atlantic      Middle Atlantic   
549937            885.0     Northeast  Middle Atlantic      Middle Atlantic   
549963           1690.0     Northeast  Middle Atlantic 

# Model Runtime

In [13]:
# Get the current datetime again
end_time = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")

# Calculate the elapsed time
elapsed_time = datetime.strptime(end_time, "%Y-%m-%d_%H-%M-%S") - datetime.strptime(start_time, "%Y-%m-%d_%H-%M-%S")

# Format the elapsed time
elapsed_seconds = elapsed_time.total_seconds()
elapsed_minutes = int(elapsed_seconds // 60)
elapsed_seconds = int(elapsed_seconds % 60)

# Print the elapsed time
print(f"The code took {elapsed_minutes} minutes and {elapsed_seconds} seconds to execute.")

The code took 1 minutes and 8 seconds to execute.
